In [ ]:
"""
COMPLETE TRADER BEHAVIOR ANALYSIS - ALL-IN-ONE SCRIPT
=====================================================
This script runs the ENTIRE analysis from start to finish with ALL outputs.

Author: Abhinav Anand
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 COMPLETE TRADER BEHAVIOR ANALYSIS")
print("="*80)
print(f"\nStarted: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# ============================================================================
# PART 1: DATA LOADING & PREPARATION
# ============================================================================

print("="*80)
print("PART 1: DATA LOADING & PREPARATION")
print("="*80)

print("\n📂 Loading datasets...")

# Load data
fear_greed = pd.read_csv('fear_greed_index.csv')
trades = pd.read_csv('historical_data.csv')

print(f"✅ Fear & Greed Index: {fear_greed.shape}")
print(f"✅ Trading Data: {trades.shape}")

# Clean data
print("\n🔧 Cleaning data...")
fear_greed['date'] = pd.to_datetime(fear_greed['date'], format='mixed', dayfirst=True)
fear_greed = fear_greed.sort_values('date').reset_index(drop=True)

trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], format='mixed', dayfirst=True)
trades['date'] = trades['Timestamp IST'].dt.date
trades['date'] = pd.to_datetime(trades['date'])
trades = trades.dropna(subset=['Closed PnL', 'Size USD', 'Execution Price'])

# Create features
trades['profitable'] = trades['Closed PnL'] > 0
trades['hour'] = trades['Timestamp IST'].dt.hour
trades['day_of_week'] = trades['Timestamp IST'].dt.day_name()

# Merge datasets
merged_df = trades.merge(
    fear_greed[['date', 'value', 'classification']],
    on='date',
    how='left'
)
merged_df = merged_df.rename(columns={
    'value': 'fear_greed_score',
    'classification': 'market_sentiment'
})

print(f"✅ Merged dataset: {len(merged_df):,} trades")

# ============================================================================
# PART 2: DEEP INSIGHTS ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("PART 2: DEEP INSIGHTS & PATTERN DISCOVERY")
print("="*80)

# INSIGHT 1: Sentiment Performance Analysis
print("\n" + "─"*80)
print("INSIGHT #1: SENTIMENT-DRIVEN PERFORMANCE PATTERNS")
print("─"*80)

sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
sentiment_analysis = merged_df.groupby('market_sentiment').agg({
    'Closed PnL': ['mean', 'median', 'std', 'sum'],
    'profitable': ['mean', 'count'],
    'Size USD': ['mean', 'sum']
}).round(2)

sentiment_analysis.columns = ['Avg_PnL', 'Median_PnL', 'PnL_StdDev', 'Total_PnL',
                              'Win_Rate', 'Trade_Count', 'Avg_Trade_Size', 'Total_Volume']
sentiment_analysis['Win_Rate'] = (sentiment_analysis['Win_Rate'] * 100).round(1)
sentiment_analysis = sentiment_analysis.reindex(sentiment_order)

print("\n📊 PERFORMANCE METRICS BY SENTIMENT:")
print(sentiment_analysis.to_string())

# Statistical significance test
extreme_fear_pnl = merged_df[merged_df['market_sentiment'] == 'Extreme Fear']['Closed PnL']
extreme_greed_pnl = merged_df[merged_df['market_sentiment'] == 'Extreme Greed']['Closed PnL']

if len(extreme_fear_pnl) > 30 and len(extreme_greed_pnl) > 30:
    t_stat, p_value = stats.ttest_ind(extreme_fear_pnl, extreme_greed_pnl)
    print(f"\n🔬 STATISTICAL TEST (Extreme Fear vs Extreme Greed):")
    print(f"   T-statistic: {t_stat:.4f}")
    print(f"   P-value: {p_value:.6f}")
    if p_value < 0.05:
        print(f"   → ✅ STATISTICALLY SIGNIFICANT (p < 0.05)")
        print(f"   → The difference is NOT due to random chance!")
    else:
        print(f"   → ⚠️ Not statistically significant (p >= 0.05)")

# Key findings
best_sentiment = sentiment_analysis['Avg_PnL'].idxmax()
worst_sentiment = sentiment_analysis['Avg_PnL'].idxmin()
performance_diff = sentiment_analysis.loc[best_sentiment, 'Avg_PnL'] - sentiment_analysis.loc[worst_sentiment, 'Avg_PnL']

print(f"\n💡 KEY FINDINGS:")
print(f"   ✅ BEST: {best_sentiment} (${sentiment_analysis.loc[best_sentiment, 'Avg_PnL']:.2f} avg PnL)")
print(f"   ❌ WORST: {worst_sentiment} (${sentiment_analysis.loc[worst_sentiment, 'Avg_PnL']:.2f} avg PnL)")
print(f"   📈 PERFORMANCE SPREAD: ${performance_diff:.2f} ({(performance_diff/abs(sentiment_analysis.loc[worst_sentiment, 'Avg_PnL']))*100:.1f}% better)")

# INSIGHT 2: Winners vs Losers Analysis
print("\n" + "─"*80)
print("INSIGHT #2: WHAT MAKES TRADES PROFITABLE?")
print("─"*80)

winners = merged_df[merged_df['profitable'] == True]
losers = merged_df[merged_df['profitable'] == False]

print(f"\n📊 TRADE DISTRIBUTION:")
print(f"   Winners: {len(winners):,} ({len(winners)/len(merged_df)*100:.1f}%)")
print(f"   Losers: {len(losers):,} ({len(losers)/len(merged_df)*100:.1f}%)")

comparison_data = {
    'Metric': ['Avg Sentiment Score', 'Avg Trade Size ($)', 'Most Common Hour', 'Total PnL ($)'],
    'Winners': [
        f"{winners['fear_greed_score'].mean():.1f}",
        f"${winners['Size USD'].mean():,.2f}",
        f"{winners['hour'].mode()[0]}:00",
        f"${winners['Closed PnL'].sum():,.2f}"
    ],
    'Losers': [
        f"{losers['fear_greed_score'].mean():.1f}",
        f"${losers['Size USD'].mean():,.2f}",
        f"{losers['hour'].mode()[0]}:00",
        f"${losers['Closed PnL'].sum():,.2f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n🔍 COMPARATIVE ANALYSIS:")
print(comparison_df.to_string(index=False))

# Sentiment difference
winner_sentiment = winners['fear_greed_score'].mean()
loser_sentiment = losers['fear_greed_score'].mean()
print(f"\n💡 SENTIMENT INSIGHT:")
if winner_sentiment > loser_sentiment:
    print(f"   → Winners trade during MORE GREEDY markets (+{winner_sentiment - loser_sentiment:.1f} points)")
else:
    print(f"   → Winners trade during MORE FEARFUL markets ({winner_sentiment - loser_sentiment:.1f} points)")

# INSIGHT 3: Temporal Patterns
print("\n" + "─"*80)
print("INSIGHT #3: OPTIMAL TRADING TIMES")
print("─"*80)

# Hourly analysis
hourly_performance = merged_df.groupby('hour').agg({
    'Closed PnL': ['mean', 'sum'],
    'profitable': 'mean',
    'Size USD': 'count'
}).round(2)

hourly_performance.columns = ['Avg_PnL', 'Total_PnL', 'Win_Rate', 'Trade_Count']
hourly_performance['Win_Rate'] = (hourly_performance['Win_Rate'] * 100).round(1)

print("\n⏰ TOP 5 BEST HOURS:")
best_hours = hourly_performance.nlargest(5, 'Avg_PnL')
print(best_hours.to_string())

print("\n⏰ TOP 5 WORST HOURS:")
worst_hours = hourly_performance.nsmallest(5, 'Avg_PnL')
print(worst_hours.to_string())

# Day of week analysis
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_performance = merged_df.groupby('day_of_week').agg({
    'Closed PnL': ['mean', 'sum'],
    'profitable': 'mean',
    'Size USD': 'count'
}).round(2)

daily_performance.columns = ['Avg_PnL', 'Total_PnL', 'Win_Rate', 'Trade_Count']
daily_performance['Win_Rate'] = (daily_performance['Win_Rate'] * 100).round(1)
daily_performance = daily_performance.reindex(day_order)

print("\n📅 PERFORMANCE BY DAY OF WEEK:")
print(daily_performance.to_string())

best_day = daily_performance['Avg_PnL'].idxmax()
best_hour = hourly_performance['Avg_PnL'].idxmax()

print(f"\n💡 TIMING RECOMMENDATION:")
print(f"   → BEST DAY: {best_day} (${daily_performance.loc[best_day, 'Avg_PnL']:.2f} avg)")
print(f"   → BEST HOUR: {best_hour}:00 (${hourly_performance.loc[best_hour, 'Avg_PnL']:.2f} avg)")
print(f"   → OPTIMAL TIME: {best_day}s at {best_hour}:00")

# INSIGHT 4: Trader Archetypes
print("\n" + "─"*80)
print("INSIGHT #4: TRADER PROFILES & SUCCESS PATTERNS")
print("─"*80)

trader_profiles = merged_df.groupby('Account').agg({
    'Closed PnL': ['sum', 'mean', 'std'],
    'Size USD': ['mean', 'sum'],
    'profitable': 'mean',
    'fear_greed_score': 'mean',
    'Timestamp IST': 'count'
}).reset_index()

trader_profiles.columns = ['Account', 'Total_PnL', 'Avg_PnL', 'PnL_Volatility',
                           'Avg_Trade_Size', 'Total_Volume', 'Win_Rate',
                           'Avg_Sentiment', 'Trade_Count']

active_traders = trader_profiles[trader_profiles['Trade_Count'] >= 10]
profitable_traders = active_traders[active_traders['Total_PnL'] > 0]

print(f"\n👥 TRADER STATISTICS:")
print(f"   Total Traders: {len(trader_profiles):,}")
print(f"   Active Traders (≥10 trades): {len(active_traders):,}")
print(f"   Profitable Traders: {len(profitable_traders):,} ({len(profitable_traders)/len(active_traders)*100:.1f}%)")

print(f"\n🏆 TOP 10 TRADERS BY PROFIT:")
top_10 = active_traders.nlargest(10, 'Total_PnL')[
    ['Account', 'Total_PnL', 'Win_Rate', 'Trade_Count', 'Avg_Trade_Size']
].copy()
top_10['Win_Rate'] = (top_10['Win_Rate'] * 100).round(1)
top_10['Total_PnL'] = top_10['Total_PnL'].round(2)
top_10['Avg_Trade_Size'] = top_10['Avg_Trade_Size'].round(2)
print(top_10.to_string(index=False))

print(f"\n💡 SUCCESS PATTERNS (Top 20% Traders):")
top_performers = active_traders.nlargest(int(len(active_traders)*0.2), 'Total_PnL')
print(f"   Average Win Rate: {top_performers['Win_Rate'].mean()*100:.1f}%")
print(f"   Average Trade Size: ${top_performers['Avg_Trade_Size'].mean():,.2f}")
print(f"   Average Trades: {top_performers['Trade_Count'].mean():.0f}")
print(f"   Preferred Sentiment: {top_performers['Avg_Sentiment'].mean():.1f}")

# INSIGHT 5: Extreme Conditions Analysis
print("\n" + "─"*80)
print("INSIGHT #5: TRADING DURING EXTREME MARKET CONDITIONS")
print("─"*80)

extreme_fear = merged_df[merged_df['fear_greed_score'] < 25]
neutral = merged_df[(merged_df['fear_greed_score'] >= 40) & (merged_df['fear_greed_score'] <= 60)]
extreme_greed = merged_df[merged_df['fear_greed_score'] > 75]

conditions = {
    'Extreme Fear (<25)': extreme_fear,
    'Neutral (40-60)': neutral,
    'Extreme Greed (>75)': extreme_greed
}

extreme_results = []
for name, data in conditions.items():
    if len(data) > 0:
        extreme_results.append({
            'Condition': name,
            'Avg_PnL': data['Closed PnL'].mean(),
            'Win_Rate': data['profitable'].mean() * 100,
            'Trade_Count': len(data),
            'Total_PnL': data['Closed PnL'].sum(),
            'Volatility': data['Closed PnL'].std()
        })

extreme_df = pd.DataFrame(extreme_results)
print("\n📊 EXTREME CONDITIONS COMPARISON:")
print(extreme_df.round(2).to_string(index=False))

best_extreme = extreme_df.loc[extreme_df['Avg_PnL'].idxmax(), 'Condition']
print(f"\n💡 EXTREME CONDITIONS STRATEGY:")
print(f"   → BEST PERFORMANCE: {best_extreme}")
print(f"   → Action: Increase position sizes during {best_extreme} conditions")

# INSIGHT 6: Coin-Specific Patterns
print("\n" + "─"*80)
print("INSIGHT #6: CRYPTOCURRENCY-SPECIFIC PERFORMANCE")
print("─"*80)

# Get major coins (most traded)
coin_stats = merged_df.groupby('Coin').agg({
    'Closed PnL': ['mean', 'sum'],
    'profitable': 'mean',
    'Size USD': 'count'
}).round(2)

coin_stats.columns = ['Avg_PnL', 'Total_PnL', 'Win_Rate', 'Trade_Count']
coin_stats['Win_Rate'] = (coin_stats['Win_Rate'] * 100).round(1)
coin_stats = coin_stats[coin_stats['Trade_Count'] >= 100]  # Only coins with 100+ trades

print(f"\n🪙 TOP 10 COINS BY PROFITABILITY (min 100 trades):")
top_coins = coin_stats.nlargest(10, 'Total_PnL')
print(top_coins.to_string())

print(f"\n🪙 TOP 10 COINS BY WIN RATE (min 100 trades):")
top_winrate_coins = coin_stats.nlargest(10, 'Win_Rate')
print(top_winrate_coins.to_string())

# ============================================================================
# PART 3: VISUALIZATIONS
# ============================================================================

print("\n" + "="*80)
print("PART 3: GENERATING VISUALIZATIONS")
print("="*80)

# Create comprehensive dashboard
fig = plt.figure(figsize=(20, 14))

# 1. Sentiment Performance Bar Chart
ax1 = plt.subplot(3, 3, 1)
colors = ['#8B0000', '#DC143C', '#808080', '#90EE90', '#006400']
sentiment_pnl = sentiment_analysis['Avg_PnL'].values
bars = ax1.bar(range(len(sentiment_order)), sentiment_pnl, color=colors, edgecolor='black', linewidth=2)
ax1.set_xticks(range(len(sentiment_order)))
ax1.set_xticklabels(sentiment_order, rotation=45, ha='right', fontsize=9, fontweight='bold')
ax1.set_ylabel('Avg PnL ($)', fontweight='bold')
ax1.set_title('💰 Performance by Sentiment', fontsize=12, fontweight='bold')
ax1.axhline(0, color='red', linestyle='--', linewidth=2)
ax1.grid(axis='y', alpha=0.3)
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'${height:.0f}', ha='center', va='bottom' if height > 0 else 'top',
            fontweight='bold', fontsize=8)

# 2. Win Rate by Sentiment
ax2 = plt.subplot(3, 3, 2)
win_rates = sentiment_analysis['Win_Rate'].values
ax2.bar(range(len(sentiment_order)), win_rates, color=colors, edgecolor='black', linewidth=2)
ax2.set_xticks(range(len(sentiment_order)))
ax2.set_xticklabels(sentiment_order, rotation=45, ha='right', fontsize=9, fontweight='bold')
ax2.set_ylabel('Win Rate (%)', fontweight='bold')
ax2.set_title('🎯 Win Rate by Sentiment', fontsize=12, fontweight='bold')
ax2.axhline(50, color='red', linestyle='--', linewidth=2, label='50%')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# 3. Hourly Performance
ax3 = plt.subplot(3, 3, 3)
hourly_pnl = hourly_performance['Avg_PnL']
ax3.plot(hourly_pnl.index, hourly_pnl.values, marker='o', linewidth=2, markersize=6, color='steelblue')
ax3.fill_between(hourly_pnl.index, hourly_pnl.values, alpha=0.3, color='steelblue')
ax3.axhline(0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Hour', fontweight='bold')
ax3.set_ylabel('Avg PnL ($)', fontweight='bold')
ax3.set_title('⏰ Hourly Performance', fontsize=12, fontweight='bold')
ax3.grid(alpha=0.3)
ax3.set_xticks(range(0, 24, 3))

# 4. Daily Performance
ax4 = plt.subplot(3, 3, 4)
daily_pnl = daily_performance['Avg_PnL']
ax4.bar(range(len(day_order)), daily_pnl.values, color='coral', edgecolor='black', linewidth=2)
ax4.set_xticks(range(len(day_order)))
ax4.set_xticklabels([d[:3] for d in day_order], fontsize=9, fontweight='bold')
ax4.set_ylabel('Avg PnL ($)', fontweight='bold')
ax4.set_title('📅 Performance by Day', fontsize=12, fontweight='bold')
ax4.axhline(0, color='red', linestyle='--', linewidth=2)
ax4.grid(axis='y', alpha=0.3)

# 5. Scatter: Sentiment vs PnL
ax5 = plt.subplot(3, 3, 5)
sample = merged_df.sample(min(5000, len(merged_df)))
scatter = ax5.scatter(sample['fear_greed_score'], sample['Closed PnL'],
                     c=sample['fear_greed_score'], cmap='RdYlGn',
                     alpha=0.4, s=10, edgecolors='none')
ax5.axhline(0, color='red', linestyle='--', linewidth=1.5)
ax5.axvline(50, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax5.set_xlabel('Fear & Greed Score', fontweight='bold')
ax5.set_ylabel('Trade PnL ($)', fontweight='bold')
ax5.set_title('📊 Sentiment vs PnL', fontsize=12, fontweight='bold')
plt.colorbar(scatter, ax=ax5, label='Sentiment')

# 6. Trader PnL Distribution
ax6 = plt.subplot(3, 3, 6)
ax6.hist(active_traders['Total_PnL'], bins=30, color='purple', edgecolor='black', alpha=0.7)
ax6.axvline(0, color='red', linestyle='--', linewidth=2, label='Break-even')
ax6.set_xlabel('Total PnL ($)', fontweight='bold')
ax6.set_ylabel('Number of Traders', fontweight='bold')
ax6.set_title('👥 Trader Profitability', fontsize=12, fontweight='bold')
ax6.legend()
ax6.grid(axis='y', alpha=0.3)

# 7. Winners vs Losers Sentiment
ax7 = plt.subplot(3, 3, 7)
sentiment_comparison = [winners['fear_greed_score'], losers['fear_greed_score']]
bp = ax7.boxplot(sentiment_comparison, labels=['Winners', 'Losers'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightgreen')
bp['boxes'][1].set_facecolor('lightcoral')
ax7.set_ylabel('Fear & Greed Score', fontweight='bold')
ax7.set_title('🔍 Sentiment: Winners vs Losers', fontsize=12, fontweight='bold')
ax7.grid(axis='y', alpha=0.3)

# 8. Trade Size Distribution
ax8 = plt.subplot(3, 3, 8)
ax8.hist([winners['Size USD'], losers['Size USD']], bins=30,
        label=['Winners', 'Losers'], color=['lightgreen', 'lightcoral'],
        edgecolor='black', alpha=0.7)
ax8.set_xlabel('Trade Size ($)', fontweight='bold')
ax8.set_ylabel('Frequency', fontweight='bold')
ax8.set_title('💵 Trade Size Distribution', fontsize=12, fontweight='bold')
ax8.legend()
ax8.set_xlim(0, merged_df['Size USD'].quantile(0.95))
ax8.grid(axis='y', alpha=0.3)

# 9. Top Coins Performance
ax9 = plt.subplot(3, 3, 9)
top_5_coins = coin_stats.nlargest(5, 'Total_PnL')
ax9.barh(range(len(top_5_coins)), top_5_coins['Total_PnL'].values,
        color='gold', edgecolor='black', linewidth=2)
ax9.set_yticks(range(len(top_5_coins)))
ax9.set_yticklabels(top_5_coins.index, fontsize=9, fontweight='bold')
ax9.set_xlabel('Total PnL ($)', fontweight='bold')
ax9.set_title('🪙 Top 5 Coins by Profit', fontsize=12, fontweight='bold')
ax9.grid(axis='x', alpha=0.3)

plt.suptitle('🎯 COMPREHENSIVE TRADER BEHAVIOR ANALYSIS DASHBOARD',
            fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('complete_analysis_dashboard.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print("\n✅ Main dashboard saved as 'complete_analysis_dashboard.png'")

# Create timing heatmap
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Win rate heatmap
pivot_winrate = merged_df.pivot_table(
    values='profitable',
    index='day_of_week',
    columns='hour',
    aggfunc='mean'
).reindex(day_order) * 100

sns.heatmap(pivot_winrate, annot=True, fmt='.0f', cmap='RdYlGn', center=50,
           cbar_kws={'label': 'Win Rate (%)'}, ax=axes[0], linewidths=0.5)
axes[0].set_title('🎯 Win Rate Heatmap (Day × Hour)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Hour of Day', fontweight='bold')
axes[0].set_ylabel('Day of Week', fontweight='bold')

# Average PnL heatmap
pivot_pnl = merged_df.pivot_table(
    values='Closed PnL',
    index='day_of_week',
    columns='hour',
    aggfunc='mean'
).reindex(day_order)

sns.heatmap(pivot_pnl, annot=True, fmt='.0f', cmap='RdYlGn', center=0,
           cbar_kws={'label': 'Avg PnL ($)'}, ax=axes[1], linewidths=0.5)
axes[1].set_title('💰 Avg PnL Heatmap (Day × Hour)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Hour of Day', fontweight='bold')
axes[1].set_ylabel('Day of Week', fontweight='bold')

plt.tight_layout()
plt.savefig('timing_heatmaps.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print("✅ Timing heatmaps saved as 'timing_heatmaps.png'")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("🎉 ANALYSIS COMPLETE - EXECUTIVE SUMMARY")
print("="*80)

print(f"""
📊 DATASET METRICS:
   • Total Trades: {len(merged_df):,}
   • Date Range: {merged_df['date'].min().date()} to {merged_df['date'].max().date()}
   • Total PnL: ${merged_df['Closed PnL'].sum():,.2f}
   • Overall Win Rate: {merged_df['profitable'].mean()*100:.1f}%

🎯 TOP INSIGHTS:

1. SENTIMENT STRATEGY:
   ✅ Best: {best_sentiment} (${sentiment_analysis.loc[best_sentiment, 'Avg_PnL']:.2f} avg PnL)
   ❌ Worst: {worst_sentiment} (${sentiment_analysis.loc[worst_sentiment, 'Avg_PnL']:.2f} avg PnL)
   → Trade primarily during {best_sentiment} conditions

2. TIMING STRATEGY:
   ⏰ Best Hour: {best_hour}:00 (${hourly_performance.loc[best_hour, 'Avg_PnL']:.2f})
   📅 Best Day: {best_day} (${daily_performance.loc[best_day, 'Avg_PnL']:.2f})
   → Schedule trading for {best_day}s at {best_hour}:00

3. TRADER SUCCESS:
   👥 {len(profitable_traders)/len(active_traders)*100:.1f}% of active traders are profitable
   🏆 Top trader made ${active_traders['Total_PnL'].max():,.2f}
   → Success correlates with {top_performers['Avg_Sentiment'].mean():.0f} avg sentiment

4. MARKET CONDITIONS:
   → Winners trade at {winner_sentiment:.1f} avg sentiment
   → Losers trade at {loser_sentiment:.1f} avg sentiment
   → Difference: {abs(winner_sentiment - loser_sentiment):.1f} points

📁 GENERATED FILES:
   1. complete_analysis_dashboard.png - Main 9-panel visualization
   2. timing_heatmaps.png - Win rate and PnL by time
   3. analysis_report.txt - Full executive summary

""")

print("="*80)
print(f"✅ COMPLETE | Duration: {(datetime.now() - datetime.now()).seconds}s")
print("="*80)

🚀 COMPLETE TRADER BEHAVIOR ANALYSIS

Started: 2025-11-08 08:52:51

PART 1: DATA LOADING & PREPARATION

📂 Loading datasets...
✅ Fear & Greed Index: (2644, 4)
✅ Trading Data: (211224, 16)

🔧 Cleaning data...
✅ Merged dataset: 211,224 trades

PART 2: DEEP INSIGHTS & PATTERN DISCOVERY

────────────────────────────────────────────────────────────────────────────────
INSIGHT #1: SENTIMENT-DRIVEN PERFORMANCE PATTERNS
────────────────────────────────────────────────────────────────────────────────

📊 PERFORMANCE METRICS BY SENTIMENT:
                  Avg_PnL  Median_PnL  PnL_StdDev   Total_PnL  Win_Rate  Trade_Count  Avg_Trade_Size  Total_Volume
market_sentiment                                                                                                  
Extreme Fear        34.54         0.0     1136.06   739110.25      37.0        21400         5349.73  1.144843e+08
Fear                54.29         0.0      935.36  3357155.44      42.0        61837         7816.11  4.833248e+08
Neutral 